In [1]:
import openai
from qdrant_client import QdrantClient

### Embedding function

In [2]:
def get_embedding(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=text,
        model=model,
    )
    return response.data[0].embedding

### Retrieval function

In [3]:
qdrant_client = QdrantClient(url="http://localhost:6333")

In [4]:
def retrieve_data(query, qdrant_client, k=5):

    query_embedding = get_embedding(query)

    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection-00",
        query=query_embedding,
        limit=k,
    )

    retrieved_context_ids = []
    retrieved_context = []
    similarity_scores = []

    for result in results.points:
        retrieved_context_ids.append(result.payload["parent_asin"])
        retrieved_context.append(result.payload["description"])
        similarity_scores.append(result.score)

    return {
        "retrieved_context_ids": retrieved_context_ids,
        "retrieved_context": retrieved_context,
        "similarity_scores": similarity_scores,
    }

In [5]:
retrieved_context = retrieve_data("What kind of earphones can I get?", qdrant_client, k=10)

In [6]:
retrieved_context

{'retrieved_context_ids': ['B0CBMPG524',
  'B0B9FTVL58',
  'B0C142QS8X',
  'B0C6K1GQCF',
  'B0BNHVLF7G',
  'B0B14HTZ59',
  'B09WCFC5D9',
  'B0B67ZFRPC',
  'B0B7495RL6',
  'B0BS15TRJ3'],
 'retrieved_context': ['Open Ear Headphones, Bluetooth 5.3 Earbuds with 60H Playtime IPX7 Waterproof Wireless Earbuds Immersive Premium Sound True Wireless Open Ear Earbuds with Earhooks for Running, Walking and Workouts 【Open-ear Design Headphones】Feature with a new generation of true open-ear wireless earbuds design, the headphones can rest gently and firmly fit your ears without entering your ear canal, which will reduce stress and hearing loss after extended wear. There is no pinching of the auricle, no blockage of the ear canal, and no pain or damage to hearing. 【Powerful Stereo Sound】Equipped with 16.2 millimeters vibrating diaphragm speaker driver, bluetooth headphones providing pure balanced audio and clarity output for all music genres with soft audio and immerse yourself in the wonderful world

### Format retrieved context function

In [7]:
def process_context(context):

    formatted_context = ""

    for id, chunk in zip(context["retrieved_context_ids"], context["retrieved_context"]):
        formatted_context += f"- {id}: {chunk}\n"

    return formatted_context

In [8]:
preprocessed_context = process_context(retrieved_context)

In [9]:
print(process_context(retrieved_context))

- B0CBMPG524: Open Ear Headphones, Bluetooth 5.3 Earbuds with 60H Playtime IPX7 Waterproof Wireless Earbuds Immersive Premium Sound True Wireless Open Ear Earbuds with Earhooks for Running, Walking and Workouts 【Open-ear Design Headphones】Feature with a new generation of true open-ear wireless earbuds design, the headphones can rest gently and firmly fit your ears without entering your ear canal, which will reduce stress and hearing loss after extended wear. There is no pinching of the auricle, no blockage of the ear canal, and no pain or damage to hearing. 【Powerful Stereo Sound】Equipped with 16.2 millimeters vibrating diaphragm speaker driver, bluetooth headphones providing pure balanced audio and clarity output for all music genres with soft audio and immerse yourself in the wonderful world of music. 【Hear Your Surroundings】Open earbuds that rest on your ears without covering them, you can hear your music and your surroundings at the same time. Whether you are cycling, walking, runn

### Create Prompt function

In [10]:
def build_prompt(preprocessed_context, question):

    prompt = f"""
You are a shopping assistant that can answer questions about the products in stock.

You will be given a question and a list of context.

Instructtions:
- You need to answer the question based on the provided context only.
- Never use word context and refer to it as the available products.

Context:
{preprocessed_context}

Question:
{question}
"""

    return prompt

In [11]:
prompt = build_prompt(preprocessed_context, "What kind of earphones can I get?")

In [12]:
print(build_prompt(preprocessed_context, "What kind of earphones can I get?"))


You are a shopping assistant that can answer questions about the products in stock.

You will be given a question and a list of context.

Instructtions:
- You need to answer the question based on the provided context only.
- Never use word context and refer to it as the available products.

Context:
- B0CBMPG524: Open Ear Headphones, Bluetooth 5.3 Earbuds with 60H Playtime IPX7 Waterproof Wireless Earbuds Immersive Premium Sound True Wireless Open Ear Earbuds with Earhooks for Running, Walking and Workouts 【Open-ear Design Headphones】Feature with a new generation of true open-ear wireless earbuds design, the headphones can rest gently and firmly fit your ears without entering your ear canal, which will reduce stress and hearing loss after extended wear. There is no pinching of the auricle, no blockage of the ear canal, and no pain or damage to hearing. 【Powerful Stereo Sound】Equipped with 16.2 millimeters vibrating diaphragm speaker driver, bluetooth headphones providing pure balanced

### Generate answer function

In [13]:
def generate_answer(prompt):

    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[{"role": "system", "content": prompt}],
        temperature=0.5,
    )

    return response.choices[0].message.content

In [14]:
print(generate_answer(prompt))

You can get the following kinds of earphones from the available products:

1. Open Ear Headphones:
   - Bluetooth 5.3 true wireless open-ear earbuds with earhooks, 60H playtime, IPX7 waterproof, suitable for running, walking, and workouts. They rest gently on your ears without entering the ear canal, allowing you to hear your surroundings while listening to music. (Product B0CBMPG524)

2. Wireless In-Ear Earbuds:
   - Bluetooth 5.3 wireless earbuds with microphone, 37H playback, LED power display, deep bass, IPX7 waterproof, ultra-light with charging case. (Product B0B9FTVL58)
   - Bluetooth 5.1 wireless earbuds with noise-cancelling mic, 30H playtime, IPX7 waterproof, ergonomic design, touch control, compatible with iPhone and Android. (Product B0C6K1GQCF)
   - Bluetooth 5.2 true wireless stereo earbuds with low latency for gaming, deep bass, touch control, 14H playtime with charging case, physical noise cancellation. (Product B09WCFC5D9)

3. Bone Conduction Headphones:
   - Wireless 

### Cimbined RAG Pipeline

In [15]:
def rag_pipeline(question, top_k=5):

    qdrant_client = QdrantClient(url="http://localhost:6333")

    retrieved_context = retrieve_data(question, qdrant_client, top_k)
    preprocessed_context = process_context(retrieved_context)
    prompt = build_prompt(preprocessed_context, question)
    answer = generate_answer(prompt)

    return answer


In [16]:
print(rag_pipeline("Can I get some cool stuff for kids?"))

Yes, you can get some cool stuff for kids from the available products. Here are two options:

1. TUNEAKE Kids Headphones (Pink) - These are over-ear, volume-limited headphones designed for kids with a comfortable adjustable headband, foldable design for easy storage, and compatibility with many devices. They are great for school, travel, or just listening to music safely.

2. QearFun Cat Earbuds for Kids (Blue) - These wired in-ear headphones come with a microphone and a lovely storage case, making them a fun and practical gift for school girls and boys.

Both options are designed specifically for kids and would make cool and useful gifts.
